# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key  
# from api_keys import geoapify_key



TypeError: the 'package' argument is required to perform a relative import for '..resources.api_keys'

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,enewetak,11.3474,162.3373,27.20,78,100,8.34,MH,1740003691
1,1,dalandzadgad,43.5708,104.4250,-9.04,52,0,1.22,MN,1740003693
2,2,lubang,13.8584,120.1240,26.48,78,52,10.52,PH,1740003694
3,3,chincha alta,-13.4099,-76.1323,23.41,81,99,2.70,PE,1740003695
4,4,sao vicente,-23.9631,-46.3919,27.77,26,46,1.34,BR,1740003696


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
humidity_per_city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
humidity_per_city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
181,181,saint-pierre,-21.3393,55.4781,24.82,73,0,1.54,RE,1740003899
239,239,tirumullaivasal,11.2333,79.8333,22.90,83,0,2.39,IN,1740003965
242,242,kattivakkam,13.2167,80.3167,25.08,80,0,1.61,IN,1740003969
251,251,castillos,-34.1667,-53.8333,21.05,71,0,4.32,UY,1740003981
269,269,kidal,18.4411,1.4078,26.00,8,0,3.39,ML,1740004002
295,295,masterton,-40.9597,175.6575,23.54,37,0,3.42,NZ,1740004032
301,301,bandrele,-12.9067,45.1914,26.86,89,0,2.57,YT,1740004040
376,376,alaghsas,17.0187,8.0168,25.69,13,0,2.58,NE,1740004131
413,413,konarka,19.9000,86.1167,24.35,77,0,4.12,IN,1740004173
434,434,tharangambadi,11.0319,79.8528,23.26,81,0,2.82,IN,1740004198


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"].tolist()
country = city_data_df["Country"].tolist()
lat = city_data_df["Lat"].tolist()
lng = city_data_df["Lng"].tolist()
humidity = city_data_df["Humidity"].tolist()


hotel_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,73,
1,tirumullaivasal,IN,11.2333,79.8333,83,
2,kattivakkam,IN,13.2167,80.3167,80,
3,castillos,UY,-34.1667,-53.8333,71,
4,kidal,ML,18.4411,1.4078,8,
5,masterton,NZ,-40.9597,175.6575,37,
6,bandrele,YT,-12.9067,45.1914,89,
7,alaghsas,NE,17.0187,8.0168,13,
8,konarka,IN,19.9000,86.1167,77,
9,tharangambadi,IN,11.0319,79.8528,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
tirumullaivasal - nearest hotel: No hotel found
kattivakkam - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
kidal - nearest hotel: No hotel found
masterton - nearest hotel: Copthorne Hotel Solway Park
bandrele - nearest hotel: Les Baobabs
alaghsas - nearest hotel: Hôtel Zagado
konarka - nearest hotel: Hotel Sun Village
tharangambadi - nearest hotel: Hotel Tamil Nadu
vikarabad - nearest hotel: Geetha Cafe


,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
1,tirumullaivasal,IN,11.2333,79.8333,83,No hotel found
2,kattivakkam,IN,13.2167,80.3167,80,No hotel found
3,castillos,UY,-34.1667,-53.8333,71,A mi gente
4,kidal,ML,18.4411,1.4078,8,No hotel found
5,masterton,NZ,-40.9597,175.6575,37,Copthorne Hotel Solway Park
6,bandrele,YT,-12.9067,45.1914,89,Les Baobabs
7,alaghsas,NE,17.0187,8.0168,13,Hôtel Zagado
8,konarka,IN,19.9000,86.1167,77,Hotel Sun Village
9,tharangambadi,IN,11.0319,79.8528,81,Hotel Tamil Nadu


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotels_per_city_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
hotels_per_city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)